## Spotify data ETL project

### Imports

In [2]:
import pandas as pd
import sqlalchemy
from sqlalchemy.orm import sessionmaker
import requests
import json
import datetime
from datetime import datetime
import sqlite3

In [4]:
# constants
DB_LOCATION = 'sqlite:///spoty_songs.sqlite'
USER_ID = 'Hristiyan'
SPOTY_TOKEN = 'BQAO4y2euytWwn9p60ynLx-JEO0aUVvm-nIkV3zs7_EMXx6yAFUyXh0YI1THqVbrxKyvynASbQ_EmVgKiAtdvWyQsXCmC5dbxCITEI5VP-aH05-a_n05YfXxx2CvfPC__v4Q0Xep7bpkfKEb1yD7XobOTq_djRSka9HgZxk6q2jcdcetIPEbjP7yqmTDra1nMLXR9A'

In [6]:
# check if current script is being run as main program
if __name__ == "__main__":
    # create the headers for the request
    headers = {
        "Accept" : "application/json",
        "Content-Type" : "application/json",
        "Authorization" : "Bearer {token}".format(token=SPOTY_TOKEN)
    }